In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = './images/train'
test_dir = './images/test'
validation_dir = './images/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze base model layers
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

Found 2224 images belonging to 2 classes.
Found 478 images belonging to 2 classes.
Found 479 images belonging to 2 classes.
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step


C:\Users\kavit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 1008s 15s/step - accuracy: 0.7809 - loss: 0.4688 - val_accuracy: 0.9420 - val_loss: 0.2140
Epoch 2/15
 1/69 ━━━━━━━━━━━━━━━━━━━━ 16:24 14s/step - accuracy: 0.9688 - loss: 0.1755

C:\Users\kavit\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


69/69 ━━━━━━━━━━━━━━━━━━━━ 190s 3s/step - accuracy: 0.9688 - loss: 0.1755 - val_accuracy: 0.9487 - val_loss: 0.2091
Epoch 3/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 1211s 18s/step - accuracy: 0.9437 - loss: 0.1825 - val_accuracy: 0.9598 - val_loss: 0.1488
Epoch 4/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 0.9062 - loss: 0.1604 - val_accuracy: 0.9598 - val_loss: 0.1478
Epoch 5/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 1210s 18s/step - accuracy: 0.9669 - loss: 0.1228 - val_accuracy: 0.9688 - val_loss: 0.1205
Epoch 6/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 210s 3s/step - accuracy: 1.0000 - loss: 0.0631 - val_accuracy: 0.9598 - val_loss: 0.1251
Epoch 7/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 1203s 17s/step - accuracy: 0.9663 - loss: 0.1124 - val_accuracy: 0.9665 - val_loss: 0.1114
Epoch 8/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 1.0000 - loss: 0.0633 - val_accuracy: 0.9665 - val_loss: 0.1122
Epoch 9/15
69/69 ━━━━━━━━━━━━━━━━━━━━ 1208s 18s/step - accuracy: 0.9760 - loss: 0.0947 - val_accuracy: 0.9621 - val_